## Reading files and creating dataframes to store all relevant tables

In [1]:
import pandas as pd
import xlrd
import numpy as np
from pandasql import sqldf
import pyxlsb
import xlwings as xw
pd.options.mode.chained_assignment = None

In [2]:
# HARDCODING the SPT file path
#spt_tool_path = 'C:\\FedEx\\FDX SPT - 2024 SAS v4.3 - ES Export_EUR v4 - Accenture MasterCopy v 0.1.xlsb'

spt_tool_path = 'C:\\FedEx\\FDX SPT - 2024 SAS v4.3 - FR Export_EUR_update_v2.xlsb'

In [3]:
# function to read tables from a single sheet with 5 given parameters

def read_tables(file_path_and_name,
                sheet_name, 
                number_of_rows_above_the_table, 
                number_of_rows_in_the_table_excluding_headers, 
                excel_column_range): # feeding the parameters required into the function definition
    
    df = pd.read_excel(file_path_and_name, 
                       engine='pyxlsb', 
                       sheet_name = sheet_name, 
                       skiprows = number_of_rows_above_the_table,  
                       nrows= number_of_rows_in_the_table_excluding_headers, 
                       usecols = excel_column_range,
                      parse_dates=[0]) # reading a table from an excel sheet
    
    return df # returning the table in pandas dataframe format

In [4]:
# reading tables listed in the Param sheet of the SPT tool

def spt_tool_tables(spt_tool_path,parameter_sheet_name):
    
    param = pd.read_excel(spt_tool_path, 
                          engine='pyxlsb', 
                          sheet_name = parameter_sheet_name) # reading the Param sheet from the SPT tool
    
    table_dict = {} # creating empty dictionary to store the tables that will be read
    sheetname=set() #creating empty set to store sheetname
    
    for i in range(len(param)):
        table_i = read_tables(spt_tool_path,
                         param.iat[i,1], # this parameter is the sheet name
                         param.iat[i,2], # this parameter is the #rows above the table
                         param.iat[i,3], # this parameter is the #rows in the table, excluding headers
                         param.iat[i,4]) # this parameter is the excel column range
        
        table_dict[str(param.iat[i,0])] = table_i # creating the key:value pair for table name: table
        sheetname.add(str(param.iat[i,1])) # Store all sheetname in set

    return table_dict,sheetname # returning the table dictionary and set

In [5]:
# HARDCODING the parameter sheet name
parameter_sheet_name = 'Param'
all_tables,all_sheetname = spt_tool_tables(spt_tool_path,parameter_sheet_name)

In [6]:
list(all_tables) # here is the entire list of tables available in the table dictionary

['master_data',
 'mark_up_premium_products',
 'domestic',
 'options',
 'collection_and_delivery',
 'covid_surcharge',
 'clearance_fee',
 'rec_pays_fee',
 'ess',
 'ooa',
 'special_handling',
 'ccf',
 '09deurope',
 'domestic:',
 'fuel',
 'exchange_rates',
 'budget_fx_rates',
 'rate_simulator_ip',
 'rate_simulator_ie_air',
 'rate_simulator_re',
 'discount_matrix_ip',
 'discount_matrix_ie_air',
 'discount_matrix_re']

In [7]:
# creating separate dataframes for each table
master_data = all_tables['master_data']
rate_simulator_ip = all_tables['rate_simulator_ip']
rate_simulator_ie_air = all_tables['rate_simulator_ie_air']
rate_simulator_re = all_tables['rate_simulator_re']
discount_matrix_ip = all_tables['discount_matrix_ip']
discount_matrix_ie_air = all_tables['discount_matrix_ie_air']
discount_matrix_re = all_tables['discount_matrix_re']
# created dataframes for only relevant tables as of now

In [8]:
# master_data.groupby(['Sce','Zoning','New Payweightband'])
master_data

,Type,Scope,…,GE Number,Inv Country Code,Rate Parent Account Nbr,Rate Parent Account Name,Account Number,Account Name,Shipment Number,...,Error old cons?,Paywgt new vs old - diff too big? (Exi buss only),Credit note?,Error Frt Rev Old? (Exi buss only),Error Frt Rev New?,Re-rate Frt Rev?,Error All-in Rev Old? (Exi buss only),Error All-in Rev New?,Re-rate All-in Rev?,Freight Rev Old inc RFP + BXT
0,Exi,In scope,…,1036170000,FR,94000,Test Account,60000,Test Account,196283856,...,False,False,False,False,False,Y,False,False,Y,53.600000
1,Exi,In scope,…,1036170000,FR,94000,Test Account,60000,Test Account,196290741,...,False,False,False,False,False,Y,False,False,Y,81.850001
2,Exi,In scope,…,1036170000,FR,94000,Test Account,60000,Test Account,196407799,...,False,False,False,False,False,Y,False,False,Y,9.360001
3,Exi,In scope,…,1036170000,FR,94000,Test Account,60000,Test Account,196699876,...,False,False,False,False,False,Y,False,False,Y,9.360001
4,Exi,In scope,…,1036170000,FR,94000,Test Account,60000,Test Account,196715273,...,False,False,False,False,False,Y,False,False,Y,13.790001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70174,NaN,NaN,NaN,1036170000,FR,1036198000,Test Account,877135000,Test Account,I41997114455,...,False,False,False,False,False,Y,False,False,Y,28.068248
70175,NaN,NaN,NaN,1036170000,FR,1036198000,Test Account,877135000,Test Account,I42000928972,...,False,False,False,False,False,Y,False,False,Y,25.741243
70176,NaN,NaN,NaN,1036170000,FR,1036198000,Test Account,877135000,Test Account,I42004133665,...,False,False,False,False,False,Y,False,False,Y,25.757220
70177,NaN,NaN,NaN,1036170000,FR,1036198000,Test Account,877135000,Test Account,I42009206843,...,False,False,False,False,False,Y,False,False,Y,28.070411


In [9]:
fdx = 'Frt Rev New '
tnt = 'Freight Rev Old inc RFP + BXT'
pivot1 = master_data.groupby(['Sce','Zoning','New Payweightband']).agg({fdx:'sum',tnt:'sum'}).reset_index()
pivot1['TNT_vs_FDX'] = pivot1[fdx] - pivot1[tnt]
pivot1['BN%'] = (pivot1['TNT_vs_FDX']*100/pivot1[tnt]).replace(np.inf, 0)
overall_bn = round(sum(pivot1['TNT_vs_FDX'])*100/sum(pivot1[tnt]),2)
print("The overall budget neutrality before running the optimization is: "+str(overall_bn)+"%\n")
input("Press Enter to continue\n")

TypeError: can only concatenate str (not "float") to str

## Level 0 Optimization starts here

In [ ]:
# defining custom function to store the binning logic of weightbands for different services
def bin_strategy(row):
    if row['Sce'] in ['IP','IE AIR']:
        # defining bins of Pay Weight Band for services IP, IE
        bins = [0, 0.5, 2.5, 5, 10, 20.5, 44.5, 70.5, 100, 300, 500, 1000, float('inf')]
        labels = ['0_0.5', '0.5_2.5', '2.5_5','5_10', '10_20.5', '20.5_44.5', '44.5_70.5', '71_100', '100_300', '300_500', '500_1000','1000_0']
    else: # defining bins of Pay Weight Band for services RE
        bins = [0, 5, 10, 20, 30, 50, 70, 71, 100, 300, 500, 1000, float('inf')]
        labels = ['0_5', '5_10', '10_20','20_30', '30_50', '50_70', '70_71', '71_100', '100_300', '300_500', '500_1000','1000_0']

    result = pd.cut([row['New Pay Weight']],bins=bins,labels=labels)
    return result[0]

In [ ]:
# subsetting the customer data table for relevant columns only
## GRI INFALTION LOGIC TO BE APPLIED ON TNT REVENUE, OPTIMIZATION SHOULD RUN POST THAT ONLY
data_columns = [
                'Frt Rev New ', #FDX
                'Freight Rev Old inc RFP + BXT', #TNT Revenue considered by FDX team in BN calculation in Frt&All-in sheet
                'New Payweightband',
                'New Pay Weight',
                'Sce',
                'Zoning',
                'Shipment Number']

data = master_data[data_columns] # subsetting only the required columns

data['key'] = data.apply(bin_strategy,axis=1) # key column is the new weightband column basically

# ordering the subsetted dataset in the following hierarchy: Service X Zone X Weightband
data.sort_values(['Sce','Zoning','key'], inplace = True)

In [ ]:
# calculating the aggregated sum of both FDX & TNT at a weightband level
discount_df = data.groupby(['Sce','Zoning','key']).agg({fdx:'sum',tnt:'sum'}).reset_index()

# calculating the overall discount for the aggregated sum for different combinations of service X zone X weightbands
discount_df['discount_weightband_level'] = ((-discount_df[fdx] + discount_df[tnt])/discount_df[fdx]).replace(np.inf, 0)

# mapping the calculated discounts to the shipment level customer data/original data
data_with_discount = pd.merge(data, discount_df[['Sce', 'Zoning', 'key','discount_weightband_level']], how='left', on=['Sce', 'Zoning', 'key'])

# Discount matrix computation

In [ ]:
def discount_matrix(customer_data,service,discount_matrix_format):
    data1 = customer_data    
    # pivoting the customer data in the format equivalent to the SPT discount matrix
    data2 = pd.pivot_table(data1[data1['Sce']==service], values = 'discount_weightband_level', index=['key'], columns = 'Zoning').reset_index()

    # renaming the columns to match the SPT tool discount matrix format
    col_dict = {}
    for col in list(data2):
        if len(col) == 1:
            col_dict[col] = 'Zone '+col

    data2.rename(columns=col_dict, inplace=True)
    
    
    if service=='RE':
        # cleaning up the original discount matrix table from the SPT tool to make it ready for the join
        discount_matrix_format.drop('Unnamed: 5', inplace = True, axis = 1)
        discount_matrix_format.rename(columns={'Table B -Weight level Adjustment - % adjustment':'FromWght',
                                           'Unnamed: 4':'ToWght'}, inplace=True)
        discount_matrix_format.drop(index=0,inplace=True)
        discount_matrix_format.reset_index(inplace=True,drop=True)
        # creating the compound joining key
        discount_matrix_format['key'] = discount_matrix_format[['FromWght','ToWght']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    
    elif service in ['IP','IE AIR']:
         # cleaning up the original discount matrix table from the SPT tool to make it ready for the join
        discount_matrix_format.drop('Unnamed: 5', inplace = True, axis = 1)
        discount_matrix_format.rename(columns={'Table B -Weight level Adjustment - % adjustment':'FromWght',
                                           'Unnamed: 4':'ToWght'}, inplace=True)
        discount_matrix_format.drop(index=0,inplace=True)
        discount_matrix_format.reset_index(inplace=True,drop=True)
        # creating the compound joining key
        discount_matrix_format['key'] = discount_matrix_format[['FromWght','ToWght']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    
    
    # performing the join
    discount_matrix_format_calculated = pd.merge(discount_matrix_format,data2,on = 'key', how = 'left', suffixes = ("_x","")) # columns from my right table will retain their original names
    # dropping duplicated columns
    cols_to_drop = [i for i in discount_matrix_format_calculated.columns if i not in discount_matrix_format.columns]
    discount_matrix_format_calculated.drop(cols_to_drop, axis=1,inplace=True)
    # correcting the order of columns in synchronization with SPT tool discount matrix
    discount_matrix_format_calculated = discount_matrix_format_calculated[discount_matrix_format.columns]
    return discount_matrix_format_calculated
    

In [ ]:
# computing discount matrix for IP
discount_matrix_ip_calculated = discount_matrix(data_with_discount,'IP',discount_matrix_ip)
discount_matrix_ip_calculated.fillna(0,inplace=True)

In [ ]:
# computing discount matrix for IE
discount_matrix_ie_air_calculated = discount_matrix(data_with_discount,'IE AIR',discount_matrix_ie_air)
discount_matrix_ie_air_calculated.fillna(0,inplace=True)

In [ ]:
# computing discount matrix for RE
discount_matrix_re_calculated = discount_matrix(data_with_discount,'RE',discount_matrix_re)
discount_matrix_re_calculated.fillna(0,inplace=True)


# Writing the discount figures back to the same SPT file

In [ ]:
def writing_discount_matrix_to_spt_file(spt_tool_path,service,discount_matrix,sheet_name,target_cell):
    xlsb_file = spt_tool_path
 
    # Open the XLSB file with xlwings
    wb = xw.Book(xlsb_file)

    # Specify the sheet where you want to write data
    sheet = wb.sheets[sheet_name]
    
    #Writig back the data frame
    data_to_write = discount_matrix[discount_matrix.columns[2:-1]].fillna(0)

    # Write data to the specific cell
    sheet.range(target_cell).value = data_to_write.values
              # ^^this is the specific cell where you want to write data
    
    # Save the changes to the XLSB file
    wb.save()

    print("Discount matrix for service "+service+" has been written to the corresponding sheet\n\n")


In [ ]:
# copying the discount matrices for all the services to the corresponding SPT sheet

writing_discount_matrix_to_spt_file(spt_tool_path,'IP',discount_matrix_ip_calculated,'7. Disc-15N (IP)','G7')
writing_discount_matrix_to_spt_file(spt_tool_path,'IE AIR',discount_matrix_ie_air_calculated,'8. Disc-48N (IE Air)','G7')
writing_discount_matrix_to_spt_file(spt_tool_path,'RE',discount_matrix_re_calculated,'9. Disc-RE-REF','G7')

# Implementing the business logic: Lighter weightbands should not be more expensive than the heavier weightbands

In [ ]:
# function to implement the copy-up logic on a single column

def transform(col): # passing each column of a dataframe as a list
    
    result = [] #list to store the resulting column with copy-up rule applied
    
    # going to check for business constraint violation from the bottom up
    for i, val in enumerate(list(col)[::-1]): # hence reversing the list of column values
        
        if np.isnan(val)==False: # handling non-null value situations
            
            if i == 0: # the last non-null value in the column will remain intact
                result.append(val)
                
            elif np.isnan(result[-1])==True: # handling the situation where the comparison is between a null and a non-null value in the column
                result.append(val)
                
            elif (val <= result[-1]) and (np.isnan(result[-1])==False): # keeping the current value intact if business logic is maintained
                result.append(val)
                
            else: # the copy-up logic implementation in action
                result.append(result[-1])
        else: # handling null value situations
            result.append(val)
  
    return (pd.Series(result[::-1])) # reversing the list again to restore to its original order

In [ ]:
# function to calculate the net rates

def copy_up(rate_simulator_df, service, discount_matrix):
    rate_simulator_df.rename(columns={'From Weight':'FromWght','To Weight':'ToWght'},inplace=True) # basic string manipulations to maintain consistency
    
    
    
    #If Fromwght more than 1000 will update towght as 1000+100
    discount_matrix['ToWght'] = discount_matrix.apply(lambda x: pd.to_numeric(x['FromWght'])+100 if float(x['FromWght'])>=1000.0 else x['ToWght'], axis=1)
    rate_simulator_df['ToWght'] = rate_simulator_df.apply(lambda x: pd.to_numeric(x['FromWght'])+100 if float(x['FromWght'])>=999.5001 else x['ToWght'], axis=1)

    # mapping the discount matrix at the corresponding rows of list rate table
    #B.FromWght=71  ,A.FromWght=70.5001
    list_rates_with_discounts_in_rows = sqldf('''
                                            SELECT A.*,B.*,
                                            CASE
                                            WHEN CAST(A.FromWght AS FLOAT) > 70.5 THEN ROUND(CAST(A.FromWght AS FLOAT))
                                            WHEN CAST(A.FromWght AS FLOAT)<70.5 THEN CAST(A.FromWght AS FLOAT)
                                            END AS FromWght_Round
                                            FROM rate_simulator_df A
                                            LEFT JOIN discount_matrix B
                                            ON CAST(FromWght_Round AS FLOAT) >= CAST(B.FromWght AS FLOAT) 
                                            AND CAST(A.ToWght AS FLOAT) <= CAST(B.ToWght AS FLOAT)
                                            ''')
    
    discounted_list_rates = rate_simulator_df.copy()

    # applying the discounts to respective columns
    for i in list_rates_with_discounts_in_rows.columns:
        for j in list_rates_with_discounts_in_rows.columns:
            if 'Zone '+i==j:
                discounted_list_rates[i] = list_rates_with_discounts_in_rows[i]*(1+list_rates_with_discounts_in_rows[j])
                
    #Copy_up logic will apply if ToWght less than 71          
    discounted_list_rates_less70_5001=discounted_list_rates[pd.to_numeric(discounted_list_rates['ToWght'])<71.00]
    discounted_list_rates_greater70_5001=discounted_list_rates[pd.to_numeric(discounted_list_rates['ToWght'])>=71.00]
    
    
    net_rates = discounted_list_rates_less70_5001.copy()

    # choosing only the zone columns
    net_rates_cols = [i for i in net_rates.columns if len(i)==1]

    # passing each of the zone columns to the transform functions
    for i in net_rates_cols:
        net_rates[i] = transform(discounted_list_rates_less70_5001[i])
        
    discounted_list_rates = pd.concat([net_rates,discounted_list_rates_greater70_5001])

    return discounted_list_rates

In [ ]:
net_rates_ip = copy_up(rate_simulator_ip,'IP',discount_matrix_ip_calculated)

In [ ]:
net_rates_ie_air = copy_up(rate_simulator_ie_air,'IE AIR',discount_matrix_ie_air_calculated)

In [ ]:
net_rates_re = copy_up(rate_simulator_re,'RE',discount_matrix_re_calculated)

In [ ]:
net_rates_re

In [ ]:
def writing_rate_simulator_to_spt_file(spt_tool_path,service,rate_simulator,sheet_name,target_cell):
    xlsb_file = spt_tool_path
 
    # Open the XLSB file with xlwings
    wb = xw.Book(xlsb_file)

    # Specify the sheet where you want to write data
    sheet = wb.sheets[sheet_name]
    
    #Writig back the data frame
    data_to_write = rate_simulator[rate_simulator.columns[2:-1]].fillna(0)

    # Write data to the specific cell
    sheet.range(target_cell).value = data_to_write.values
              # ^^this is the specific cell where you want to write data
    
    # Save the changes to the XLSB file
    wb.save()

    print("Rate simulator for service "+service+" has been written to the corresponding sheet\n\n")

In [ ]:
writing_rate_simulator_to_spt_file(spt_tool_path,'IP',net_rates_ip,[x for x in all_sheetname if x.startswith('Rate Simulator IP')][0],'AA5')
writing_rate_simulator_to_spt_file(spt_tool_path,'IE AIR',net_rates_ie_air,[x for x in all_sheetname if x.startswith('Rate Simulator IE')][0],'AA5')
writing_rate_simulator_to_spt_file(spt_tool_path,'RE',net_rates_re,[x for x in all_sheetname if x.startswith('Rate Simulator RE-REF')][0],'AF5')

In [ ]:
import datetime
master_test=master_data.copy()

master_test['Shipment Date'] = pd.to_numeric(master_data['Shipment Date'], errors='coerce')

# Function to convert Excel serial date number to date format
convert_to_date = lambda excel_date: (datetime.datetime(1899, 12, 31) + datetime.timedelta(days=excel_date - 1)).strftime('%m/%d/%Y') if excel_date > 0 else None
 
# Applying the lambda function to the 'excel_date' column
master_test['Shipment Date'] = master_test['Shipment Date'].apply(convert_to_date)

master_test[(master_test['Shipment Date']>'10/03/2022') &  (master_test['Shipment Date']<'12/03/2022')]

# END OF CODE

In [ ]:
# data['key'] =   np.where((data['New Pay Weight']<=0.5)&(data['Sce'].isin(['IP','IE AIR'])), '0_0.5',
#                 np.where((data['New Pay Weight']<=2.5)&(data['Sce'].isin(['IP','IE AIR'])), '0.5_2.5',
#                 np.where((data['New Pay Weight']<=5)&(data['Sce'].isin(['IP','IE AIR'])), '2.5_5',
#                 np.where((data['New Pay Weight']<=10)&(data['Sce'].isin(['IP','IE AIR'])), '5_10',
#                 np.where((data['New Pay Weight']<=20.5)&(data['Sce'].isin(['IP','IE AIR'])), '10_20.5',
#                 np.where((data['New Pay Weight']<=44.5)&(data['Sce'].isin(['IP','IE AIR'])), '20.5_44.5',
#                 np.where((data['New Pay Weight']<=70.5)&(data['Sce'].isin(['IP','IE AIR'])), '44.5_70.5',
#                 np.where((data['New Pay Weight']<=100)&(data['Sce'].isin(['IP','IE AIR'])), '71_100',
#                 np.where((data['New Pay Weight']<=300)&(data['Sce'].isin(['IP','IE AIR'])), '100_300',
#                 np.where((data['New Pay Weight']<=500)&(data['Sce'].isin(['IP','IE AIR'])), '300_500',
#                 np.where((data['New Pay Weight']<=1000)&(data['Sce'].isin(['IP','IE AIR'])), '500_1000',
#                 np.where((data['New Pay Weight']>1000)&(data['Sce'].isin(['IP','IE AIR'])), '1000_0',
#                 np.where((data['New Pay Weight']<=5)&(data['Sce']=='RE'), '0_5',
#                 np.where((data['New Pay Weight']<=10)&(data['Sce']=='RE'), '5_10',
#                 np.where((data['New Pay Weight']<=20)&(data['Sce']=='RE'), '10_20',
#                 np.where((data['New Pay Weight']<=30)&(data['Sce']=='RE'), '20_30',
#                 np.where((data['New Pay Weight']<=50)&(data['Sce']=='RE'), '30_50',
#                 np.where((data['New Pay Weight']<=70)&(data['Sce']=='RE'), '50_70',
#                 np.where((data['New Pay Weight']<=71)&(data['Sce']=='RE'), '70_71',
#                 np.where((data['New Pay Weight']<=100)&(data['Sce']=='RE'), '71_100',
#                 np.where((data['New Pay Weight']<=300)&(data['Sce']=='RE'), '100_300',
#                 np.where((data['New Pay Weight']<=500)&(data['Sce']=='RE'), '300_500',
#                 np.where((data['New Pay Weight']<=1000)&(data['Sce']=='RE'), '500_1000',
#                 np.where((data['New Pay Weight']>1000)&(data['Sce']=='RE'), '1000_0','NaN'))))))))))))))))))))))))

In [ ]:

# bins = {
#     'IP': [0, 0.5, 2.5, 5, 10, 20.5, 44.5, 70.5, 100, 300, 500, 1000, float('inf')],
#     'IE AIR': [0, 0.5, 2.5, 5, 10, 20.5, 44.5, 70.5, 100, 300, 500, 1000, float('inf')],
#     'RE': [0, 5, 10, 20, 30, 50, 70, 71, 100, 300, 500, 1000, float('inf')]
# }

# # Use pandas cut to create the 'Binned_Age' column
# data['key'] = data.apply(lambda row: pd.cut([row['New Pay Weight']], bins[row['Sce']], labels=False), axis=1)

# # Map the labels to the corresponding bins
# bin_labels = {
#     'IP': ['0_0.5', '0.5_2.5', '2.5_5','5_10', '10_20.5', '20.5_44.5', '44.5_70.5', '71_100', '100_300', '300_500', '500_1000','1000_0'],
#     'IE AIR': ['0_0.5', '0.5_2.5', '2.5_5','5_10', '10_20.5', '20.5_44.5', '44.5_70.5', '71_100', '100_300', '300_500', '500_1000','1000_0'],
#     'RE': ['0_5', '5_10', '10_20','20_30', '30_50', '50_70', '70_71', '71_100', '100_300', '300_500', '500_1000','1000_0']
# }
# data['key'] = data.apply(lambda row: bin_labels[row['Sce']][row['New Pay Weight']], axis=1)

# if data['Sce'].isin(['IP','IE AIR']):
#     data['key'] = pd.cut(x=data[data['Sce'].isin(['IP','IE AIR'])]['New Pay Weight'], 
#                          bins=[0, 0.5, 2.5, 5, 10, 20.5, 44.5, 70.5, 100, 300, 500, 1000, 10000],
#                          labels=['0_0.5', '0.5_2.5', '2.5_5','5_10', '10_20.5', '20.5_44.5', '44.5_70.5', '71_100', '100_300', '300_500', '500_1000','1000_0'])

# data[data['Sce']=='RE']['key'] = pd.cut(x=data[data['Sce']=='RE']['New Pay Weight'], 
#                      bins=[0, 5, 10, 20, 30, 50, 70, 71, 100, 300, 500, 1000, 10000],
#                      labels=['0_5', '5_10', '10_20','20_30', '30_50', '50_70', '70_71', '71_100', '100_300', '300_500', '500_1000','1000_0'])
# data[data['Sce'].isin(['IP','IE AIR'])]['key'] = data[data['Sce'].isin(['IP','IE AIR'])].assign


In [ ]:
#         data['key'] =   np.where((data['New Pay Weight']<=0.5)&(data['Sce'].isin(['IP','IE AIR'])), '0_0.5',
#                         np.where((data['New Pay Weight']<=2.5)&(data['Sce'].isin(['IP','IE AIR']), '0.5_2.5',
#                         np.where((data['New Pay Weight']<=5)&(data['Sce'].isin(['IP','IE AIR']), '2.5_5',
#                         np.where((data['New Pay Weight']<=10)&(data['Sce'].isin(['IP','IE AIR']), '5_10',
#                         np.where((data['New Pay Weight']<=20.5)&(data['Sce'].isin(['IP','IE AIR']), '10_20.5',
#                         np.where((data['New Pay Weight']<=44.5)&(data['Sce'].isin(['IP','IE AIR']), '20.5_44.5',
#                         np.where((data['New Pay Weight']<=70.5)&(data['Sce'].isin(['IP','IE AIR']), '44.5_70.5',
#                         np.where((data['New Pay Weight']<=100)&(data['Sce'].isin(['IP','IE AIR']), '71_100',
#                         np.where((data['New Pay Weight']<=300)&(data['Sce'].isin(['IP','IE AIR']), '100_300',
#                         np.where((data['New Pay Weight']<=500)&(data['Sce'].isin(['IP','IE AIR']), '300_500',
#                         np.where((data['New Pay Weight']<=1000)&(data['Sce'].isin(['IP','IE AIR']), '500_1000',
#         data['key'] =   np.where(data['New Pay Weight']<=5, '0_5',
#                         np.where(data['New Pay Weight']<=10, '5_10',
#                         np.where(data['New Pay Weight']<=20, '10_20',
#                         np.where(data['New Pay Weight']<=30, '20_30',
#                         np.where(data['New Pay Weight']<=50, '30_50',
#                         np.where(data['New Pay Weight']<=70, '50_70',
#                         np.where(data['New Pay Weight']<=71, '70_71',
#                         np.where(data['New Pay Weight']<=100, '71_100',
#                         np.where(data['New Pay Weight']<=300, '100_300',
#                         np.where(data['New Pay Weight']<=500, '300_500',
#                         np.where(data['New Pay Weight']<=1000, '500_1000','1000_0')))))))))))
    
# max_val = data['New Pay Weight'].max()

In [ ]:
# # pivoting the customer data in the format equivalent to the SPT discount matrix
# data2 = pd.pivot_table(data1[data1['Sce']=='IE AIR'], values = 'discount_weightband_level', index=['New Payweightband'], columns = 'Zoning').reset_index()
# # creating required columns for joining with SPT discount matrix
# data2['FromWght'] = ['0',	'0.5',	'2.5',	'5',	'10',	'20.5',	'44.5',	'71',	'100',	'300']
# data2['ToWght'] =   ['0.5',	'2.5',	'5',	'10',	'20.5',	'44.5',	'70.5',	'100',	'300',	'500']
# # creating the compound joining key
# data2['key'] = data2[['FromWght','ToWght']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

# # renaming the columns to match the SPT tool discount matrix format
# col_dict = {}
# for col in list(data2):
#     if len(col) == 1:
#         col_dict[col] = 'Zone '+col
        
# data2.rename(columns=col_dict, inplace=True)
# data2.head()

# # cleaning up the original discount matrix table from the SPT tool to make it ready for the join
# discount_matrix_ip.drop('Unnamed: 5', inplace = True, axis = 1)
# discount_matrix_ip.rename(columns={'Table B -Weight level Adjustment - % adjustment':'FromWght',
#                                    'Unnamed: 4':'ToWght'}, inplace=True)
# discount_matrix_ip.drop(index=0,inplace=True)
# discount_matrix_ip.reset_index(inplace=True,drop=True)
# # creating the compound joining key
# discount_matrix_ip['key'] = discount_matrix_ip[['FromWght','ToWght']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
# discount_matrix_ip.head()

# # performing the join
# discount_matrix_ip_calculated2 = pd.merge(discount_matrix_ip,data2,on = 'key', how = 'left', suffixes = ("_x","")) # columns from my right table will retain their original names
# # dropping duplicated columns
# cols_to_drop = [i for i in discount_matrix_ip_calculated2.columns if i not in discount_matrix_ip.columns]
# discount_matrix_ip_calculated2.drop(cols_to_drop, axis=1,inplace=True)
# # correcting the order of columns in synchronization with SPT tool discount matrix
# discount_matrix_ip_calculated2 = discount_matrix_ip_calculated2[discount_matrix_ip.columns]
# discount_matrix_ip_calculated2

In [ ]:
# for index,row in data.iterrows():
#     if (row[0]!=0)&(row[1]!=0):
#         data.loc[index,'discount%'] = (-row[1] + row[0])/row[1]
#     else:
#         data.loc[index,'discount%'] = float("Nan")

# data['bn_level1%'] = (data['Frt Rev New ']*(1 - data['discount%']/100) - data['Freight Revenue'])/data['Freight Revenue']*100

# mean_discount = data.groupby(['Sce','Zoning','New Payweightband'], as_index=False, sort=False).mean()

# discount_matrix_ip = pd.pivot_table(mean_discount[mean_discount['Service']=='Express'], values = 'discount%', index=['New Payweightband'], columns = 'Zoning').reset_index()

In [ ]:
# dups = data.groupby('Shipment Number').filter(lambda x: len(x) > 1)
# dups_list = list(set(dups['Shipment Number']))

# data_unique = data[~data['Shipment Number'].isin(dups_list)]
# len(data_unique)

# data['bn_level0%'] = (data['Frt Rev New '] - data['Freight Revenue'])/data['Freight Revenue']*100

In [ ]:
# mean_discount[(mean_discount['Service']=='Economy')&(mean_discount['Zoning']=='A')&
#               (mean_discount['New Pay Weight']>=0.5)&(mean_discount['New Pay Weight']<=2.5)
#              ]['discount%'].values[0]

# for index,row in discount_matrix_ip.iterrows():
#     discount_matrix_ip.loc[index,'Zone A'] = mean_discount[(mean_discount['Service']=='Express')&(mean_discount['Zoning']=='A')&
#                                                            (mean_discount['New Pay Weight']>=row[0])&(mean_discount['New Pay Weight']<=row[1])
#                                                           ]['discount%'].values[0]

In [ ]:
# data['Shipment Number'].nunique()
# disc = data.groupby('Shipment Number').filter(lambda x: len(x) > 1)
# list(set(disc['Shipment Number']))
# data[data['Shipment Number']==614872888]

In [ ]:
# # Replace 'your_file.xlsx' with the path to your Excel file
# file_path = 'C:\\Users\\rishav.ganguly\\Downloads\\FedEx\\FDX SPT - 2024 SAS v4.3 - GB Export_GBP_Accenture (2)\\FDX SPT - 2024 SAS v4.3 - GB Export_GBP_Accenture.xlsb',
# # Replace 'Sheet1' with the name of the Excel tab
# sheet_name = '2. Add. Info'

# # Define the ranges for each table in the Excel tab
# # Each range should be specified as 'A1:B3', 'C5:E10', etc., where 'A1', 'B3', etc., define the table boundaries
# table_ranges = {
#     'CCF': 'O:Q',  # Replace with your desired range
#     'Clearance Fee': 'K:M',  # Replace with your desired range
#     # Add more tables as needed
# }

# # Create an empty dictionary to store the tables
# tables = {}

# # Read each specified range into a DataFrame and store it in the 'tables' dictionary
# with pd.ExcelFile(file_path) as xls:
#     for table_name, range_str in table_ranges.items():
#         df = pd.read_excel(xls, engine='pyxlsb', sheet_name=sheet_name, header=0, usecols=range_str)
#         tables[table_name] = df

# # Now, 'tables' is a dictionary containing separate DataFrames for each specified table in the Excel tab
# # You can access each table using its name as the key
# # For example, to access 'Table1':
# ccf = tables['CCF']
# ccf

# # Repeat the above process for each table you want to extract from the Excel tab

In [ ]:
# # #import all the libraries
# from office365.runtime.auth.authentication_context import AuthenticationContext
# from office365.sharepoint.client_context import ClientContext
# from office365.sharepoint.files.file import File 
# import io
# # # import pandas as pd

# # # #target url taken from sharepoint and credentials
# url = 'https://ts.accenture.com/:x:/s/FedEx55/EWyUT6mK60BKnneUTCACVg0B2EgdeiP7XbJnWsKy3LLWxQ?e=gMVnYL'
# username = 'rishav.ganguly@accenture.com'
# password = 'Puku@0707wb'

# ctx_auth = AuthenticationContext(url)
# if ctx_auth.acquire_token_for_user(username, password):
#   ctx = ClientContext(url, ctx_auth)
#   web = ctx.web
#   ctx.load(web)
#   ctx.execute_query()
#   print("Authentication successful")

# response = File.open_binary(ctx, url)

# # #save data to BytesIO stream
# bytes_file_obj = io.BytesIO()
# bytes_file_obj.write(response.content)
# bytes_file_obj.seek(0) #set file object to start

# # # #read excel file and each sheet into pandas dataframe 
# # df = pd.read_excel(bytes_file_obj, engine='pyxlsb', sheet_name = 'Parameters')

In [ ]:
# import re

# text = 'c 2.51-5kg'

# # Define a regular expression pattern to match decimal numbers
# pattern = r'(\d+\.\d+)'

# # Use re.findall to find all matching numbers in the text
# numeric_list = re.findall(pattern, text)

# # Convert the list of matched strings to a list of floats
# numeric_list = [round(float(num),1) for num in numeric_list]

# print(numeric_list)

# #############################################################################################################
# import re

# def extract_numeric_range(text):
#     # Define a regular expression pattern to match decimal numbers
#     pattern = r'(\d+\.\d+)-(\d+\.\d+)'

#     # Use re.search to find the first match in the text
#     match = re.search(pattern, text)

#     # Check if a match was found
#     if match:
#         start = float(match.group(1))
#         end = float(match.group(2))
#         # Round to the nearest tenth
#         return [round(start, 1), round(end, 1)]
#     else:
#         return None

# # Test with your examples
# examples = [
#     "b 0.51-2.5kg",
#     "c 2.51-5kg",
#     "f 20.51-44.5kg",
#     "g 44.51-70.5kg"
# ]

# for example in examples:
#     numeric_range = extract_numeric_range(example)
#     if numeric_range is not None:
#         print(f"Input text: '{example}'")
#         print(f"Output numeric list: {numeric_range}")
#         print()
#     else:
#         print(f"No numeric range found in '{example}'")